## Reading in the dataset

In [1]:
import os, sys
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import operator

assert os.path.exists('./emails.train.csv'), "[Dataset File Not Found] Please download dataset first."

In [25]:
# Read in csv file as dataframe
df = pd.read_csv('./emails.train.csv')
zf = pd.read_csv('./emails.test.csv')

# Show a snippet of dataset.
print(df.head())
print(np.sum(zf['spam']))

spamappend = 0
hamappend = 0
for i in df["spam"]:
    if i == 1:
        spamappend += 1
    if i == 0:
        hamappend +=1

probspam = float(spamappend)/(spamappend+hamappend)
print(probspam)
spam = sum(df["spam"])
ham =  len(df["spam"])-sum(df["spam"])
probspam = float(spam)/(spam+ham)
print(probspam)

   id                                               text  spam
0   0  Subject: naturally irresistible your corporate...     1
1   2  Subject: unbelievable new homes made easy  im ...     1
2   3  Subject: 4 color printing special  request add...     1
3   4  Subject: do not have money , get software cds ...     1
4   5  Subject: great nnews  hello , welcome to medzo...     1
415
0.23700571997015668
0.23700571997


## Text cleaning

In [26]:
def cleaning_text(dataframe):
    print('Cleaning text...')
    import re
    
    for i, line in enumerate(dataframe['text']):
        
        # remove special characters using regex
        line = re.sub(r'[^\w]', ' ', line).lower()
        # remove numeric characters using regex
        line = re.sub("(^|\W)\d+($|\W)", " ", line)
           
        # remove words of two characters or less
        words = line.split()
        resultwords = []
        
        for word in words:
            if len(word) > 2:
                resultwords.append(word)
                
        newline = ' '.join(resultwords)

        dataframe.loc[i, 'text'] = newline

    return dataframe
        
print('========= Clean training data ==========')
# Read in training data
df = pd.read_csv('./emails.train.csv')

# Do cleaning
df = cleaning_text(df)

# Save as new file
df.to_csv('emails_clean.train.csv')

print('========= Clean testing data ==========')
# Read in testing data
df = pd.read_csv('./emails.test.csv')

# Do cleaning
df = cleaning_text(df)

# Save as new file
df.to_csv('emails_clean.test.csv')

print('Cleaned the data')

========= Clean training data ==========
Cleaning text...
========= Clean testing data ==========
Cleaning text...
Cleaned the data


## Retrieve word frequency

In [27]:
# get word frequency data for both train data
def word_frequency_train(dataframe):
    # for w in dataframe.str.lower():
#     create a dataframe of all words
    
    datasetwords = {}
    
    for line in dataframe["text"]:
        words = line.split()
        for word in words:
            if word in datasetwords:
                datasetwords[word] += 1
            else:
                datasetwords[word] = 1
    
    return datasetwords

# get word frequency data for mails
def word_frequency_mail(mail):
    words = mail.split()
    datasetwords = {}
    for word in words:
        if word in datasetwords:
            datasetwords[word] += 1
        else:
            datasetwords[word] = 1
    
    return datasetwords    

## Model

In [43]:
# freqsword = frequency word in spam, freqhword is frequency word in ham
def NaiveBayesian(probspam,freqsword,freqhword,totalfreqs,totalfreqh):
    
#   probability of a mail being ham 
    probham = 1-probspam
    
#   calculate probability of word being in spam or word being in spam
    probwordspam = float(freqsword)/(totalfreqs)
    probwordham = float(freqhword)/(totalfreqh)
    bayes = (float(probwordspam*probspam)/(probwordspam*probspam + probwordham*probham))


#       equation for correcting the probability where s =3 means that meaning that the learned dictionary
#       must contain more than 3 messages with the word
    s = 3
    correctedbayes = float(s*probspam+freqsword*bayes)/(s+freqsword)
    
#   if the spamicity of the word is around 0.5 return none since the probability of the word being ham or spam
#   is too similar so we evaluate only spammicities of <=0.2 and >= 0.8
    
    if correctedbayes >= 0.8 or bayes <=0.2:
        return correctedbayes

    else:
        return None

    
    


In [44]:

# Here we call our Naive Bayesian performer with the data we read in from train and test and check
# if the test mail is spam or not
def CallBae(freqspam,freqham,mail,probspam,totalfreqs,totalfreqh):
    spamchecklist = []
    
    for word in mail:
    
#     if word in spam and ham check its frequency and perform the Bayesian as many times as needed
        if word in freqspam and word in freqham:
            for i in range(mail[word]):
                freqsword = freqspam[word]
                freqhword = freqham[word]
                spamchecklist.append(NaiveBayesian(probspam,freqsword,freqhword,totalfreqs,totalfreqh))

    #   if the word is in neither spam or ham we continue the word iteration
        elif word not in freqspam and word not in freqham:
            continue

    #  we initialize freqsword as a tiny number here cause else the Bayesian would not be able to pick up that 
    #  the word is in ham if the word never appears in spam but does appear in ham
        elif word in freqham and word not in freqspam:
            for i in range(mail[word]):
                freqsword = 10**(-8)
                freqhword = freqham[word]
                spamchecklist.append(NaiveBayesian(probspam,freqsword,freqhword,totalfreqs,totalfreqh))

            
#   Clean up the spamchecklist
    spamchecklist = np.array(spamchecklist)
    spamchecklist = spamchecklist[spamchecklist != np.array(None)]
    
################################################################
#    Again zou eigenlijk correct meoten zijn maar werkt niet (dit selecteert van de data alleen de 10 beste entries)
#     try:
#         spamchecklist = np.take(spamchecklist,(np.argpartition(abs(0.5-spamchecklist), -10)[-10:]))
#     except:
#         spamchecklist = spamchecklist
#  #####################################################################   

    
#   Count all probabilities in spamchecklist and calculate the overal probability of the mail being spam or ham
    multiplicationsum = np.prod(spamchecklist)
    ones = np.ones(spamchecklist.shape)
    pminusonesum = np.prod(ones-spamchecklist)
    
#   Make sure we won't have conflicts with machine precision for if the mail is almost certainly spam we get very
#   small numbers which add to 0 under machine precision
    if pminusonesum + multiplicationsum ==0:
        finaljudge = 1
    else:   
        finaljudge = multiplicationsum/(multiplicationsum+pminusonesum)
        
#########################################
# DIT WERKT SOMEHOW BETER maar theoretisch onjuist dus niet super handig te gebruiken (voor de overal probability te berekenen)
#     finaljudge = sum(spamchecklist/len(spamchecklist))
###############################################
    
#   Return False if spam
    if finaljudge > 0.5:
        return True
#   Return True if ham
    if finaljudge < 0.5:
        return False

In [37]:
# Takes as input frequency table of spam words, frequency table of ham words,the mail we have to check,
# probability mail is spam, total frequency of words in spam and total frequency of words  in  ham
def Classifier(freqspam,freqham,test,probspam,totalfreqs,totalfreqh,train):
    iddict = {}
    spamdict = {}
    idlist = []
    spamlist = [] 
    
# loop through all the mails in the test and perform naive bayesian and add id number to the list
    for i in range(len(test)):
        idlist.append(test['id'][i])
        mail = test['text'][i]        
        mail = word_frequency_mail(mail)
    
#         if we get True from CallBae mail is spam and we add it to the spam dictionary
        if CallBae(freqspam,freqham,mail,probspam,totalfreqs,totalfreqh):
            spamlist.append(1)

            freqspam = combine_dicts(freqspam,mail)
            totalfreqs = sum(freqspam.values())      
    
#             If we get False from Callbae mail is added to ham dictionary and defined as ham
        else:
            spamlist.append(0)
            freqham = combine_dicts(freqham,mail)
            totalfreqh = sum(freqham.values())
                   
    
#   create our submission with per id the indicator whether the mail is spam or not (1 or 0)
    spamdict['spam'] = spamlist
    iddict['id'] = idlist
    spamdict = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in spamdict.items()]))
    iddict = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in iddict.items()]))
    submission = pd.concat([iddict,spamdict],axis = 1)
    submission.set_index('id',inplace=True)
    submission.to_csv('submission.csv')
    return submission,idlist

In [41]:
# function for combining dictionaries
def combine_dicts(a, b, op=operator.add):
    return dict(a.items() + b.items() +[(k, op(a[k], b[k])) for k in set(b) & set(a)])

In [42]:
# intialize initials
train = pd.read_csv('./emails_clean.train.csv')
test  = pd.read_csv('./emails_clean.test.csv')
hamdata = train[train.spam == 0]
spamdata = train[train.spam == 1]
freqham = word_frequency_train(hamdata)
freqspam = word_frequency_train(spamdata)
totalfreqs = sum(freqspam.values())
totalfreqh = sum(freqham.values())
spam = sum(train["spam"])
ham =  len(train["spam"])-sum(train["spam"])
probspam = 0.5
##################################################################
# DEZE HEURISTIEK WERKT VOOR GENE METER DUS HOUD PROBSPAM op 0.5
# probspam = float(spam)/float(ham+spam)
###############################################################

# Call our  model
submission,idlist = Classifier(freqspam,freqham,test,probspam,totalfreqs,totalfreqh,train)


# MEAN CONSEQUENTIAL EVALUATER:

ms = 0
test.set_index('id',inplace=True)
for i in idlist:
    try:      
        if test["spam"][i] != submission["spam"][i]:
            ms +=1
    except:
        continue

MCE = 1./submission.shape[0]*ms

print(MCE)

TypeError: unsupported operand type(s) for +: 'dict_items' and 'dict_items'

## Plotting